In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from gym import spaces

In [3]:
import sys
sys.path.append('/Users/jaime/Documents/MachineLearning/LuxAI/LuxAI-RLlib/')
sys.path.append('/Users/jaime/Documents/MachineLearning/LuxAI/LuxPrivate/')

In [4]:
from multilux.lux_env import LuxEnv
from multilux.lux_game import LuxGame

Loading environment football failed: No module named 'gfootball'


In [5]:
import helpers
from my_multilux_interface import MyInterface

In [7]:
# (2) Register environment --------------------------------------------------
from ray.tune.registry import register_env
from multilux.lux_env import LuxEnv

def env_creator(env_config):
    
    configuration = env_config.get(configuration, {})
    debug = env_config.get(debug, False)
    interface = env_config.get(interface, MyInterface)
    agents = env_config.get(agents, (None, "simple_agent"))
    
    return LuxEnv(configuration, debug,
                  interface=interface,
                  agents=agents,
                  train=True)

register_env("multilux", env_creator)

# (3) Instantiate agent ------------------------------------------------------
import random
from ray.rllib.agents import ppo

config = {
    "env_config": {},
    "multiagent": {
        "policies": {
            # the first tuple value is None -> uses default policy
            "unit-1": (None, 
                       MyInterface.obs_spaces["unit"], 
                       MyInterface.act_spaces["unit"], 
                       {"gamma": 0.85}),
            "unit-2": (None, 
                       MyInterface.obs_spaces["unit"], 
                       MyInterface.act_spaces["unit"], 
                       {"gamma": 0.99}),
            "citytile": (None, 
                         MyInterface.obs_spaces["citytile"], 
                         MyInterface.act_spaces["citytile"], 
                         {}),
        },
        "policy_mapping_fn":
            lambda agent_id:
                "citytile"  # Citytiles always have the same policy
                if agent_id.startswith("ct_")
                else random.choice(["unit-1", "unit-2"])  # Randomly choose from unit policies
    },
}

trainer = ppo.PPOTrainer(env=LuxEnv, config=config)

# (4) Train away -------------------------------------------------------------
while True:
    print(trainer.train())

/Users/jaime/opt/anaconda3/envs/LuxAI/lib/python3.8/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


RayActorError: The actor died because of an error raised in its creation task, [36mray::RolloutWorker.__init__()[39m (pid=35751, ip=192.168.20.8)
ray._private.memory_monitor.RayOutOfMemoryError: More than 95% of the memory on node Jaime-MBP is used (15.32 / 16.0 GB). The top 10 memory consumers are:

PID	MEM	COMMAND
11608	0.99GiB	/Applications/Brave Browser.app/Contents/Frameworks/Brave Browser Framework.framework/Versions/92.1.
35752	0.93GiB	ray::IDLE
35751	0.93GiB	ray::IDLE
35735	0.85GiB	/Users/jaime/opt/anaconda3/envs/LuxAI/bin/python -m ipykernel_launcher -f /Users/jaime/Library/Jupyt
25012	0.67GiB	/Applications/PyCharm CE.app/Contents/MacOS/pycharm
583	0.3GiB	/Applications/Brave Browser.app/Contents/MacOS/Brave Browser
35744	0.24GiB	/Users/jaime/opt/anaconda3/envs/LuxAI/bin/python -u /Users/jaime/opt/anaconda3/envs/LuxAI/lib/python
35746	0.2GiB	/Users/jaime/opt/anaconda3/envs/LuxAI/bin/python -u /Users/jaime/opt/anaconda3/envs/LuxAI/lib/python
1319	0.19GiB	/Applications/Toggl Tr

(pid=35752) 2021-09-15 13:31:48,712	ERROR worker.py:428 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=35752, ip=192.168.20.8)
(pid=35752) ray._private.memory_monitor.RayOutOfMemoryError: More than 95% of the memory on node Jaime-MBP is used (15.32 / 16.0 GB). The top 10 memory consumers are:
(pid=35752) 
(pid=35752) PID	MEM	COMMAND
(pid=35752) 11608	0.99GiB	/Applications/Brave Browser.app/Contents/Frameworks/Brave Browser Framework.framework/Versions/92.1.
(pid=35752) 35752	0.93GiB	ray::IDLE
(pid=35752) 35751	0.93GiB	ray::IDLE
(pid=35752) 35735	0.85GiB	/Users/jaime/opt/anaconda3/envs/LuxAI/bin/python -m ipykernel_launcher -f /Users/jaime/Library/Jupyt
(pid=35752) 25012	0.67GiB	/Applications/PyCharm CE.app/Contents/MacOS/pycharm
(pid=35752) 583	0.3GiB	/Applications/Brave Browser.app/Contents/MacOS/Brave Browser
(pid=35752) 35744	0.24GiB	/Users/jaime/opt/anaconda3/envs/LuxAI/bin/python -u /Users/ja

## DEBUGGING

In [14]:
from kaggle_environments import make

# Make environment
env = make("lux_ai_2021", configuration={
    "seed": 562124210, 
#     "loglevel": 2, 
#     "annotations": True
}, debug=True)

# Training agent in first position (player 1) against "simple_agent"
trainer = env.train([None, "simple_agent"])

observation = trainer.reset()


game = LuxGame(observation)
game_state = game.update(observation)

# game_state.players[0].cities['c_1'].citytiles[0].pos
# game_state.players[0].cities['c_1'].citytiles[0].__class__.__name__.lower()
# game_state.players[0].units[0].id
# observation

In [24]:
my_actors = game.get_team_actors(teams=(game.player_id,), flat=True)
obs = helpers.get_observation_as_tensor(game_state)